In [1]:
%matplotlib inline
import matplotlib as mpl
plt = mpl.pyplot
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.autolimit_mode'] = 'round_numbers'
mpl.rcParams['axes.xmargin'] = 0
mpl.rcParams['axes.ymargin'] = 0
import numpy as np
import utils

In [2]:
# training_features, training_labels, validation_features, validation_labels = \
#     utils.get_training_data(0.2, onehot=False, standardize=False)
# print(training_features.shape, training_labels.shape,
#      validation_features.shape, validation_labels.shape)

In [3]:
# Tf-Idf transform
# from sklearn.feature_extraction.text import TfidfTransformer
# tfidf = TfidfTransformer()
# training_features = tfidf.fit_transform(training_features)
# validation_features = tfidf.transform(validation_features)

In [4]:
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers.core import Dense, Activation, Dropout

def create_model(dense=(160,), activation='sigmoid', dropout=0.7):
    model = Sequential()
    for d in dense:
        model.add(Dense(d, input_shape=(1000,)))
        model.add(BatchNormalization())
        model.add(Activation(activation))
        model.add(Dropout(dropout))
    model.add(Dense(1))
    model.add(BatchNormalization())
    model.add(Activation('sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

from keras.wrappers.scikit_learn import KerasClassifier
class KerasClassifierMod(KerasClassifier):
    def fit(self, x, y, sample_weight, **kwargs):
        return super().fit(x, y, sample_weight=sample_weight, **kwargs)

Using TensorFlow backend.


In [5]:
nn = KerasClassifierMod(build_fn=create_model, epochs=1, batch_size=32, verbose=1)

In [6]:
from sklearn.ensemble import AdaBoostClassifier
n_estimators = 50
learning_rate = 0.1
ada_nn = AdaBoostClassifier(base_estimator=nn, n_estimators=n_estimators, 
                        learning_rate=learning_rate)

In [7]:
# ada_nn.fit(training_features, training_labels)

In [8]:
from sklearn.metrics import zero_one_loss
def plot_ada_hist(ada, title):
    ada_acc = np.zeros((n_estimators,))
    for i, pred_labels in enumerate(ada.staged_predict(validation_features)):
        ada_acc[i] = zero_one_loss(pred_labels, validation_labels)

    ada_acc_train = np.zeros((n_estimators,))
    for i, pred_labels in enumerate(ada.staged_predict(training_features)):
        ada_acc_train[i] = zero_one_loss(pred_labels, training_labels)

    plt.figure(dpi=300)
    plt.plot(np.arange(n_estimators) + 1, ada_acc, label='AdaBoost Test Loss')
    plt.plot(np.arange(n_estimators) + 1, ada_acc_train, label='AdaBoost Train Loss')
    plt.semilogy()
    plt.legend()
    plt.title(title)
    plt.show()
    plt.clf()

In [9]:
# plot_ada_hist(ada_nn, 'AdaBoost with neural network\nfinal accuracy %.4f'
#              %ada_nn.score(validation_features, validation_labels))

In [10]:
# Now the final training
training_features1, training_labels1, _, _ = \
    utils.get_training_data(0, onehot=False, standardize=False)
print(training_features1.shape, training_labels1.shape)

(20000, 1000) (20000,)


In [11]:
test_features1 = utils.get_test_data(standardize=False)
print(test_features1.shape)

(10000, 1000)


In [ ]:
# Tf-Idf transform
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()
training_features1 = tfidf.fit_transform(training_features1)
test_features1 = tfidf.transform(test_features1)

In [ ]:
nn = KerasClassifierMod(build_fn=create_model, epochs=1, batch_size=32, verbose=1)
from sklearn.ensemble import AdaBoostClassifier
n_estimators = 80
learning_rate = 0.05
ada_nn1 = AdaBoostClassifier(base_estimator=nn, n_estimators=n_estimators, 
                        learning_rate=learning_rate)
ada_nn1.fit(training_features1, training_labels1)

Epoch 1/1
20000/20000 [==============================] - 1s 62us/step
Epoch 1/1
20000/20000 [==============================] - 1s 64us/step
Epoch 1/1
20000/20000 [==============================] - 1s 66us/step
Epoch 1/1
20000/20000 [==============================] - 1s 67us/step
Epoch 1/1
20000/20000 [==============================] - 1s 69us/step
Epoch 1/1
20000/20000 [==============================] - 1s 74us/step
Epoch 1/1
20000/20000 [==============================] - 2s 78us/step
Epoch 1/1
20000/20000 [==============================] - 2s 76us/step
Epoch 1/1
20000/20000 [==============================] - 2s 82us/step
Epoch 1/1
20000/20000 [==============================] - 2s 88us/step
Epoch 1/1
20000/20000 [==============================] - 2s 83us/step
Epoch 1/1
20000/20000 [==============================] - 2s 87us/step
Epoch 1/1
20000/20000 [==============================] - 2s 89us/step
Epoch 1/1
20000/20000 [==============================] - 2s 89us/step
Epoch 1/1
20000/2000

In [ ]:
test_labels1 = ada_nn1.predict(test_features1)
utils.save_prediction(test_labels1, 'data/test_labels_ada_nn.txt')